In [0]:
import time
import math
import os
import os.path
import numpy as np
import pandas as pd
import seaborn as sns
from pprint import pprint
import io
import requests
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction import DictVectorizer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split


In [0]:
df=pd.read_csv('assistments.csv')
df

,problemlog_id,assignment_id,sequence_id,user_id,ARRS,assistment_id,problem_id,sequence_opp,assignment_opp,ARRS-check(oneMeansFirstWeek-TwomeansOneweekLater-threemeanstwoweeksafterthat),correct,hint_count,attempt_count,first_response_time,overlap_time,cluster,pretest,posttest,avg_initial_three_correctness,avg_initial_#problems_to_master,avg_first_reassess
0,40790855,297282,5959,75063,0,34310,53621,1,1,1,0,0,1,13260,13353,0,0,1,0.671795,6.323529,0.684211
1,40790864,297282,5959,75063,0,34289,53537,2,2,1,1,0,1,4820,4820,0,0,1,0.671795,6.323529,0.684211
2,40790866,297282,5959,75063,0,34295,53561,3,3,1,1,0,1,8340,8340,0,0,1,0.671795,6.323529,0.684211
3,40790867,297282,5959,75063,0,34338,53733,4,4,1,1,0,1,10758,10758,0,0,1,0.671795,6.323529,0.684211
4,40897310,299510,5959,75063,0,34329,53697,5,1,2,1,0,0,10,10,1,0,1,0.671795,6.323529,0.684211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25400,41038106,301286,15675,93241,1,121297,231946,1,1,1,1,0,1,44759,44759,0,1,0,0.630769,5.470588,0.666667
25401,41038288,301286,15675,93241,1,121263,231911,2,2,1,0,0,5,55244,70970,0,1,0,0.630769,5.470588,0.666667
25402,41038533,301286,15675,93241,1,121220,231868,3,3,1,1,0,1,20336,20336,0,1,0,0.630769,5.470588,0.666667
25403,41038618,301286,15675,93241,1,121261,231909,4,4,1,1,0,1,12483,12483,0,1,0,0.630769,5.470588,0.666667


In [0]:
#want question with low guess. Slip?
#from skill dataset, predicting using "correct" as testing value does not seem quite right.
#but for now, we can analyze if we can predict whether a tudents answers it right or not

In [0]:
url = "https://drive.google.com/uc?export=download&id=0B3f_gAH-MpBmUmNJQ3RycGpJM0k"
s = requests.get(url).content
skill = pd.read_csv(io.StringIO(s.decode('latin')))

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
skill.head()

,order_id,assignment_id,user_id,assistment_id,problem_id,original,correct,attempt_count,ms_first_response,tutor_mode,answer_type,sequence_id,student_class_id,position,type,base_sequence_id,skill_id,skill_name,teacher_id,school_id,hint_count,hint_total,overlap_time,template_id,answer_id,answer_text,first_action,bottom_hint,opportunity,opportunity_original
0,33022537,277618,64525,33139,51424,1,1,1,32454,tutor,algebra,5948,13241,126,MasterySection,5948,1.0,Box and Whisker,22763,73,0,3,32454,30799,NaN,26,0,NaN,1,1.0
1,33022709,277618,64525,33150,51435,1,1,1,4922,tutor,algebra,5948,13241,126,MasterySection,5948,1.0,Box and Whisker,22763,73,0,3,4922,30799,NaN,55,0,NaN,2,2.0
2,35450204,220674,70363,33159,51444,1,0,2,25390,tutor,algebra,5948,11816,22,MasterySection,5948,1.0,Box and Whisker,22763,73,0,3,42000,30799,NaN,88,0,NaN,1,1.0
3,35450295,220674,70363,33110,51395,1,1,1,4859,tutor,algebra,5948,11816,22,MasterySection,5948,1.0,Box and Whisker,22763,73,0,3,4859,30059,NaN,41,0,NaN,2,2.0
4,35450311,220674,70363,33196,51481,1,0,14,19813,tutor,algebra,5948,11816,22,MasterySection,5948,1.0,Box and Whisker,22763,73,3,4,124564,30060,NaN,65,0,0.0,3,3.0


In [0]:
X_train, X_test, y_train, y_test = train_test_split(skill.drop(columns=['correct']), skill['correct'], test_size=0.20, random_state=0)

In [0]:
X_train['y'] = y_train
X_train

,order_id,assignment_id,user_id,assistment_id,problem_id,original,attempt_count,ms_first_response,tutor_mode,answer_type,sequence_id,student_class_id,position,type,base_sequence_id,skill_id,skill_name,teacher_id,school_id,hint_count,hint_total,overlap_time,template_id,answer_id,answer_text,first_action,bottom_hint,opportunity,opportunity_original,y
184835,23200471,265375,78953,53499,93470,1,2,64954,tutor,algebra,7229,12438,6,MasterySection,7118,77.0,Finding Percents,23907,76,3,3,81298,52571,NaN,3.6,0,1.0,8,8.0,0
374541,29017855,270967,82162,36950,58136,0,1,8328,tutor,choose_1,9656,12455,25,MasterySection,5918,NaN,NaN,49076,5177,0,2,8328,30050,87307.0,NaN,0,NaN,21,NaN,1
196122,27908880,268663,84385,37150,58707,1,2,17205,tutor,choose_1,5979,12696,13,MasterySection,5979,79.0,Proportion,55938,5255,0,5,19620,34874,88504.0,NaN,0,NaN,3,3.0,0
163341,32752764,275289,84872,46436,84624,1,1,112086,tutor,algebra,6851,12359,65,MasterySection,6851,70.0,Percent Of,33567,76,0,4,112086,46397,NaN,32.8,0,NaN,3,3.0,1
342558,24035566,265533,78048,32038,49005,0,1,5929,tutor,choose_1,5921,12350,1,MasterySection,5921,NaN,NaN,11446,1645,0,2,5929,30244,72115.0,NaN,0,NaN,9,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359783,31906874,271255,79034,44376,80219,0,3,7376,tutor,algebra,6468,12351,174,MasterySection,6468,NaN,NaN,42999,2770,2,2,39315,42219,NaN,26,0,1.0,159,NaN,0
358083,30952943,270298,79022,41731,73574,0,1,8202,tutor,algebra,6094,12351,158,MasterySection,6094,NaN,NaN,42999,2770,0,4,8202,30422,NaN,25-Mar,0,NaN,78,NaN,1
152315,20977354,262741,78915,34596,54111,1,1,68359,tutor,algebra,5964,12352,18,MasterySection,5964,67.0,Multiplication Fractions,42999,2770,0,2,68359,29862,NaN,15/16,0,NaN,2,2.0,1
117952,27001373,262723,79017,39561,66484,1,1,5252,tutor,choose_1,6040,12352,9,MasterySection,6040,49.0,Ordering Positive Decimals,42999,2770,0,4,5252,30634,105382.0,NaN,0,NaN,22,22.0,1


In [0]:
df_x = X_train.drop(columns=['y'])
data = df_x.to_dict('records')
vec = DictVectorizer()
data = vec.fit_transform(data).toarray()

X = data
y = y_train
clff = DecisionTreeClassifier(max_depth=3)
clff = clff.fit(X, y)
y_pred = clff.predict(X)

In [0]:
#checking that all problem_id in df is in skill


# df_problem = df['problem_id'].unique()
# skill_problem = skill['problem_id'].unique()
# df_problem.isin(skill_problem)


joined = df.set_index('problem_id').join(skill.set_index('problem_id'), on = 'problem_id', rsuffix = 'right_suf')
joined

# joined = pd.merge(df,skill[['problem_id', 'base_sequence_id', 'skill_id', 'skill_name']], on = 'problem_id', how = 'left')
# joined

#weird situation: without setting index, the problem_id will not match up. But now the problem_id do match up, there are too many rows

,problemlog_id,assignment_id,sequence_id,user_id,ARRS,assistment_id,sequence_opp,assignment_opp,ARRS-check(oneMeansFirstWeek-TwomeansOneweekLater-threemeanstwoweeksafterthat),correct,hint_count,attempt_count,first_response_time,overlap_time,cluster,pretest,posttest,avg_initial_three_correctness,avg_initial_#problems_to_master,avg_first_reassess,order_id,assignment_idright_suf,user_idright_suf,assistment_idright_suf,original,correctright_suf,attempt_countright_suf,ms_first_response,tutor_mode,answer_type,sequence_idright_suf,student_class_id,position,type,base_sequence_id,skill_id,skill_name,teacher_id,school_id,hint_countright_suf,hint_total,overlap_timeright_suf,template_id,answer_id,answer_text,first_action,bottom_hint,opportunity,opportunity_original
problem_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
53621,40790855,297282,5959,75063,0,34310,1,1,1,0,0,1,13260,13353,0,0,1,0.671795,6.323529,0.684211,33652911.0,279792.0,71148.0,34310.0,1.0,1.0,1.0,6031.0,tutor,algebra,10529.0,12623.0,1.0,MasterySection,5959.0,58.0,Addition Whole Numbers,24116.0,76.0,0.0,0.0,6031.0,30524.0,NaN,23,0.0,NaN,4.0,4.0
53621,40790855,297282,5959,75063,0,34310,1,1,1,0,0,1,13260,13353,0,0,1,0.671795,6.323529,0.684211,33652177.0,279792.0,71235.0,34310.0,1.0,1.0,1.0,3515.0,tutor,algebra,10529.0,12623.0,1.0,MasterySection,5959.0,58.0,Addition Whole Numbers,24116.0,76.0,0.0,0.0,3515.0,30524.0,NaN,23,0.0,NaN,3.0,3.0
53621,40790855,297282,5959,75063,0,34310,1,1,1,0,0,1,13260,13353,0,0,1,0.671795,6.323529,0.684211,35034198.0,277494.0,71821.0,34310.0,1.0,0.0,1.0,8923.0,tutor,algebra,5959.0,13241.0,2.0,MasterySection,5959.0,58.0,Addition Whole Numbers,22763.0,73.0,0.0,0.0,9079.0,30524.0,NaN,232,0.0,NaN,3.0,3.0
53621,40790855,297282,5959,75063,0,34310,1,1,1,0,0,1,13260,13353,0,0,1,0.671795,6.323529,0.684211,29018938.0,271290.0,78414.0,34310.0,1.0,0.0,1.0,45576.0,tutor,algebra,9795.0,12455.0,26.0,MasterySection,5959.0,58.0,Addition Whole Numbers,49076.0,5177.0,0.0,0.0,45589.0,30524.0,NaN,33,0.0,NaN,12.0,12.0
53621,40790855,297282,5959,75063,0,34310,1,1,1,0,0,1,13260,13353,0,0,1,0.671795,6.323529,0.684211,29017216.0,271290.0,78423.0,34310.0,1.0,1.0,1.0,11752.0,tutor,algebra,9795.0,12455.0,26.0,MasterySection,5959.0,58.0,Addition Whole Numbers,49076.0,5177.0,0.0,0.0,11752.0,30524.0,NaN,23,0.0,NaN,15.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231946,41038106,301286,15675,93241,1,121297,1,1,1,1,0,1,44759,44759,0,1,0,0.630769,5.470588,0.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
231911,41038288,301286,15675,93241,1,121263,2,2,1,0,0,5,55244,70970,0,1,0,0.630769,5.470588,0.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
231868,41038533,301286,15675,93241,1,121220,3,3,1,1,0,1,20336,20336,0,1,0,0.630769,5.470588,0.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
print(len(df['assistment_id'].unique()))
print(len(df['problem_id'].unique()))
print(len(df['sequence_id'].unique())) #short. order appears?
print(len(df['assignment_id'].unique())) #as long as the DF. Seems like unique
# print(len(df['problemlog_id'].unique())) : 25405

3165
3165
33
817


In [0]:
skill.head()

,order_id,assignment_id,user_id,assistment_id,problem_id,original,correct,attempt_count,ms_first_response,tutor_mode,answer_type,sequence_id,student_class_id,position,type,base_sequence_id,skill_id,skill_name,teacher_id,school_id,hint_count,hint_total,overlap_time,template_id,answer_id,answer_text,first_action,bottom_hint,opportunity,opportunity_original
0,33022537,277618,64525,33139,51424,1,1,1,32454,tutor,algebra,5948,13241,126,MasterySection,5948,1.0,Box and Whisker,22763,73,0,3,32454,30799,NaN,26,0,NaN,1,1.0
1,33022709,277618,64525,33150,51435,1,1,1,4922,tutor,algebra,5948,13241,126,MasterySection,5948,1.0,Box and Whisker,22763,73,0,3,4922,30799,NaN,55,0,NaN,2,2.0
2,35450204,220674,70363,33159,51444,1,0,2,25390,tutor,algebra,5948,11816,22,MasterySection,5948,1.0,Box and Whisker,22763,73,0,3,42000,30799,NaN,88,0,NaN,1,1.0
3,35450295,220674,70363,33110,51395,1,1,1,4859,tutor,algebra,5948,11816,22,MasterySection,5948,1.0,Box and Whisker,22763,73,0,3,4859,30059,NaN,41,0,NaN,2,2.0
4,35450311,220674,70363,33196,51481,1,0,14,19813,tutor,algebra,5948,11816,22,MasterySection,5948,1.0,Box and Whisker,22763,73,3,4,124564,30060,NaN,65,0,0.0,3,3.0


In [0]:
#problem type, only original=1 main problems? , hint_count, overlap_time